In [4]:
# =========================
# PROJET 2 OFFLINE — VERSION JUPYTER
# Volatilité & VaR + Backtest + SQL + Excel + Graphiques
#
# Fichier requis dans le même dossier que ce notebook:
# - prices.csv  (colonnes: Date, AAPL, GLD, GSPC)
#
# Dépendances (à lancer si besoin):
# !pip install pandas numpy matplotlib openpyxl
# =========================

import os
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# 0) PARAMÈTRES
# -------------------------
CSV_PATH = "prices.csv"  # dans le même dossier que le notebook

TICKERS = ["GSPC", "AAPL", "GLD"]          # noms EXACTS des colonnes du CSV
WEIGHTS = np.array([0.50, 0.30, 0.20])     # doit sommer à 1

VAR_LEVELS = [0.95, 0.99]
ROLLING_WINDOW = 250
VOL_WINDOWS = [30, 90]

OUT_DIR = "outputs"
FIG_DIR = os.path.join(OUT_DIR, "figures")
DB_PATH = os.path.join(OUT_DIR, "risk.db")
REPORT_PATH = os.path.join(OUT_DIR, "report_risk.xlsx")


# -------------------------
# 1) UTILS
# -------------------------
def ensure_dirs():
    os.makedirs(OUT_DIR, exist_ok=True)
    os.makedirs(FIG_DIR, exist_ok=True)

def check_weights():
    if len(WEIGHTS) != len(TICKERS):
        raise ValueError("WEIGHTS et TICKERS doivent avoir la même longueur.")
    s = float(np.sum(WEIGHTS))
    if not np.isclose(s, 1.0, atol=1e-8):
        raise ValueError(f"Les poids doivent sommer à 1. Somme actuelle = {s}")

def var_from_returns(returns: pd.Series, level: float) -> float:
    alpha = 1.0 - level
    return float(np.quantile(returns.dropna().values, alpha))

def annualized_vol(returns: pd.Series) -> float:
    return float(returns.dropna().std(ddof=1) * np.sqrt(252))


# -------------------------
# 2) LOAD OFFLINE (CSV)
# -------------------------
def load_prices_offline():
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(
            f"Fichier introuvable: {CSV_PATH}\n"
            "➡️ Mets prices.csv dans le même dossier que ton notebook."
        )

    df = pd.read_csv(CSV_PATH)

    if "Date" not in df.columns:
        raise ValueError("Le CSV doit contenir une colonne 'Date'.")

    for t in TICKERS:
        if t not in df.columns:
            raise ValueError(f"Colonne manquante dans le CSV: {t}")

    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date")

    prices = df.set_index("Date")[TICKERS].copy()
    prices = prices.replace([np.inf, -np.inf], np.nan).dropna(how="any")

    if len(prices) < (ROLLING_WINDOW + 60):
        raise ValueError(
            f"Pas assez de lignes: {len(prices)}.\n"
            f"Il faut idéalement >= {ROLLING_WINDOW + 60} jours."
        )

    return prices


# -------------------------
# 3) PRIX → RENDEMENTS → PORTEFEUILLE
# -------------------------
def compute_returns(prices: pd.DataFrame):
    rets = prices.pct_change().dropna()
    ret_port = (rets * WEIGHTS).sum(axis=1)
    ret_port.name = "ret_port"
    return rets, ret_port


# -------------------------
# 4) METRICS: VOL + VaR
# -------------------------
def compute_metrics(rets: pd.DataFrame, ret_port: pd.Series):
    metrics = []
    for t in TICKERS:
        metrics.append({
            "scope": "asset",
            "name": t,
            "mean_daily": float(rets[t].mean()),
            "vol_daily": float(rets[t].std(ddof=1)),
            "vol_annual": annualized_vol(rets[t]),
        })

    metrics.append({
        "scope": "portfolio",
        "name": "PORTFOLIO",
        "mean_daily": float(ret_port.mean()),
        "vol_daily": float(ret_port.std(ddof=1)),
        "vol_annual": annualized_vol(ret_port),
    })

    metrics_df = pd.DataFrame(metrics)

    var_df = pd.DataFrame([
        {"method": "Historical", "level": lvl, "VaR": var_from_returns(ret_port, lvl)}
        for lvl in VAR_LEVELS
    ])

    vol_roll = pd.DataFrame(index=ret_port.index)
    for w in VOL_WINDOWS:
        vol_roll[f"vol_{w}d"] = ret_port.rolling(w).std()

    return metrics_df, var_df, vol_roll


# -------------------------
# 5) BACKTEST
# -------------------------
def backtest_var(ret_port: pd.Series):
    bt = pd.DataFrame({"ret_port": ret_port})
    for lvl in VAR_LEVELS:
        alpha = 1.0 - lvl
        bt[f"VaR_{int(lvl*100)}"] = ret_port.rolling(ROLLING_WINDOW).quantile(alpha)
        bt[f"breach_{int(lvl*100)}"] = (bt["ret_port"] < bt[f"VaR_{int(lvl*100)}"]).astype(int)
    return bt.dropna()

def summarize_backtest(bt: pd.DataFrame):
    rows = []
    n = len(bt)
    for lvl in VAR_LEVELS:
        col = f"breach_{int(lvl*100)}"
        rows.append({
            "level": lvl,
            "expected_breach_rate": 1.0 - lvl,
            "observed_breach_rate": float(bt[col].mean()),
            "n_days_tested": n,
            "n_breaches": int(bt[col].sum())
        })
    return pd.DataFrame(rows)


# -------------------------
# 6) STRESS TESTS
# -------------------------
def stress_tests(ret_port: pd.Series):
    equity_weight = float(WEIGHTS[0] + WEIGHTS[1])  # GSPC + AAPL
    return pd.DataFrame([
        {"scenario": "EquityShock_-5% (approx)", "impact": -0.05 * equity_weight},
        {"scenario": "WorstDay_Historical", "impact": float(ret_port.min())},
        {"scenario": "VolSpike_x1.5 (VaR95)", "impact": var_from_returns(ret_port * 1.5, 0.95)},
        {"scenario": "VolSpike_x1.5 (VaR99)", "impact": var_from_returns(ret_port * 1.5, 0.99)},
    ])


# -------------------------
# 7) SQL (SQLite)
# -------------------------
def save_to_sqlite(rets: pd.DataFrame, bt: pd.DataFrame):
    if os.path.exists(DB_PATH):
        os.remove(DB_PATH)

    conn = sqlite3.connect(DB_PATH)

    rets_long = rets.copy()
    rets_long.index = rets_long.index.astype(str)
    rets_long = rets_long.reset_index().rename(columns={"index": "Date"})
    rets_long = rets_long.melt(id_vars="Date", var_name="ticker", value_name="ret")
    rets_long.to_sql("returns", conn, if_exists="replace", index=False)

    bt_sql = bt.copy()
    bt_sql.index = bt_sql.index.astype(str)
    bt_sql = bt_sql.reset_index().rename(columns={"index": "Date"})
    bt_sql.to_sql("portfolio", conn, if_exists="replace", index=False)

    q_asset = """
    SELECT ticker, AVG(ret) AS mean_ret
    FROM returns
    GROUP BY ticker
    ORDER BY ticker;
    """

    q_month = """
    SELECT substr(Date, 1, 7) AS month,
           SUM(breach_95) AS breaches_95,
           SUM(breach_99) AS breaches_99
    FROM portfolio
    GROUP BY month
    ORDER BY month;
    """

    asset_stats_sql = pd.read_sql_query(q_asset, conn)
    breaches_by_month_sql = pd.read_sql_query(q_month, conn)

    conn.close()
    return asset_stats_sql, breaches_by_month_sql


# -------------------------
# 8) GRAPHIQUES
# -------------------------
def make_plots(ret_port: pd.Series, vol_roll: pd.DataFrame, bt: pd.DataFrame):
    # Histogram
    plt.figure()
    plt.hist(ret_port.values, bins=80, density=True)
    plt.title("Portfolio daily returns distribution (offline)")
    plt.xlabel("daily return")
    plt.ylabel("density")
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "portfolio_returns_hist.png"), dpi=160)
    plt.close()

    # Rolling vol
    plt.figure()
    for c in vol_roll.columns:
        plt.plot(vol_roll.index, vol_roll[c], label=c)
    plt.title("Rolling volatility (portfolio)")
    plt.xlabel("date")
    plt.ylabel("rolling std")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "rolling_vol.png"), dpi=160)
    plt.close()

    # Backtest VaR95
    lvl = 0.95
    var_col = "VaR_95"
    plt.figure()
    plt.plot(bt.index, bt["ret_port"], label="ret_port")
    plt.plot(bt.index, bt[var_col], label=var_col, linestyle="--")
    plt.title(f"Backtest: returns vs {var_col} (rolling {ROLLING_WINDOW}d)")
    plt.xlabel("date")
    plt.ylabel("daily return")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, "backtest_var95.png"), dpi=160)
    plt.close()


# -------------------------
# 9) REPORT EXCEL
# -------------------------
def export_excel(prices, rets, ret_port, metrics_df, var_df, vol_roll,
                 bt, bt_summary, stress_df, asset_stats_sql, breaches_by_month_sql):
    with pd.ExcelWriter(REPORT_PATH, engine="openpyxl") as writer:
        prices.to_excel(writer, sheet_name="Prices")
        rets.to_excel(writer, sheet_name="Returns")
        ret_port.to_frame().to_excel(writer, sheet_name="Portfolio_Returns")
        metrics_df.to_excel(writer, sheet_name="Metrics", index=False)
        var_df.to_excel(writer, sheet_name="VaR", index=False)
        vol_roll.to_excel(writer, sheet_name="RollingVol")
        bt.to_excel(writer, sheet_name="Backtest")
        bt_summary.to_excel(writer, sheet_name="Backtest_Summary", index=False)
        stress_df.to_excel(writer, sheet_name="StressTests", index=False)
        asset_stats_sql.to_excel(writer, sheet_name="SQL_AssetStats", index=False)
        breaches_by_month_sql.to_excel(writer, sheet_name="SQL_BreachesByMonth", index=False)


# -------------------------
# RUN (JUPYTER)
# -------------------------
def run_project():
    ensure_dirs()
    check_weights()

    print("Working directory:", os.getcwd())
    print("CSV expected at:", os.path.abspath(CSV_PATH))

    print("\n1) Load OFFLINE prices...")
    prices = load_prices_offline()
    print("   Prices shape:", prices.shape)

    print("2) Compute returns & portfolio...")
    rets, ret_port = compute_returns(prices)

    print("3) Compute metrics...")
    metrics_df, var_df, vol_roll = compute_metrics(rets, ret_port)

    print("4) Backtest rolling VaR...")
    bt = backtest_var(ret_port)
    bt_summary = summarize_backtest(bt)

    print("5) Stress tests...")
    stress_df = stress_tests(ret_port)

    print("6) Save SQLite + SQL summaries...")
    asset_stats_sql, breaches_by_month_sql = save_to_sqlite(rets, bt)

    print("7) Make plots...")
    make_plots(ret_port, vol_roll, bt)

    print("8) Export Excel...")
    export_excel(
        prices, rets, ret_port,
        metrics_df, var_df, vol_roll,
        bt, bt_summary, stress_df,
        asset_stats_sql, breaches_by_month_sql
    )

    print("\n✅ DONE (OFFLINE JUPYTER)")
    print("- Excel report:", os.path.abspath(REPORT_PATH))
    print("- SQLite DB:", os.path.abspath(DB_PATH))
    print("- Figures folder:", os.path.abspath(FIG_DIR))

    # Affiche quelques résultats dans le notebook
    display(metrics_df)
    display(var_df)
    display(bt_summary)
    display(stress_df)

# Lance le projet
run_project()


Working directory: C:\Users\sharu\Documents\Projet finance
CSV expected at: C:\Users\sharu\Documents\Projet finance\prices.csv

1) Load OFFLINE prices...
   Prices shape: (600, 3)
2) Compute returns & portfolio...
3) Compute metrics...
4) Backtest rolling VaR...
5) Stress tests...
6) Save SQLite + SQL summaries...
7) Make plots...
8) Export Excel...

✅ DONE (OFFLINE JUPYTER)
- Excel report: C:\Users\sharu\Documents\Projet finance\outputs\report_risk.xlsx
- SQLite DB: C:\Users\sharu\Documents\Projet finance\outputs\risk.db
- Figures folder: C:\Users\sharu\Documents\Projet finance\outputs\figures


,scope,name,mean_daily,vol_daily,vol_annual
0,asset,GSPC,0.001278,0.011916,0.189159
1,asset,AAPL,0.000502,0.019498,0.309514
2,asset,GLD,0.001143,0.010039,0.159362
3,portfolio,PORTFOLIO,0.001018,0.008715,0.138346


,method,level,VaR
0,Historical,0.95,-0.012945
1,Historical,0.99,-0.017927


,level,expected_breach_rate,observed_breach_rate,n_days_tested,n_breaches
0,0.95,0.05,0.054286,350,19
1,0.99,0.01,0.011429,350,4


,scenario,impact
0,EquityShock_-5% (approx),-0.040000
1,WorstDay_Historical,-0.021638
2,VolSpike_x1.5 (VaR95),-0.019418
3,VolSpike_x1.5 (VaR99),-0.026891
